In [22]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from helper_functions import *

### Preparation

In [2]:
# Read the database and eliminate the repeated index column
df = pd.read_csv("profit_x_y_scaling.csv")
df = df.drop("Unnamed: 0", axis =1)
pd.set_option("display.max_columns", None)
df

title_x    title_y  duration_x  budget_x  LANGUAGE_English_x  \
0     tt0448115  tt0437086    0.421053  0.315457                   1   
1     tt0451279  tt0437086    0.488722  0.470032                   1   
2     tt0451279  tt0448115    0.488722  0.470032                   1   
3     tt0493405  tt0437086    0.180451  0.078864                   1   
4     tt0493405  tt0448115    0.180451  0.078864                   1   
...         ...        ...         ...       ...                 ...   
9445  tt2873282  tt2482856    0.481203  0.217666                   1   
9446  tt2873282  tt2492344    0.481203  0.217666                   1   
9447  tt2873282  tt2531334    0.481203  0.217666                   1   
9448  tt2873282  tt2674358    0.481203  0.217666                   1   
9449  tt2873282  tt2731308    0.481203  0.217666                   1   

      LANGUAGE_Chinese_x  LANGUAGE_Korean_x  PC_Marvel Studios_x  \
0                      0                  0                    0   
1                      1                  0                    0   
2                      1                  0                    0   
3                      0                  0                    0   
4                      0                  0                    0   
...                  ...                ...                  ...   
9445                   0                  0                    0   
9446                   0                  0                    0   
9447                   0                  0                    0   
9448                   0                  0                    0   
9449                   0                  0                    0   

      PC_Columbia Pictures_x  PC_Universal Pictures_x  PC_Warner Bros_x  \
0                          0                        0                 1   
1                          0                        0                 1   
2                          0                        0                 1   
3                          0                        0                 0   
4                          0                        0                 0   
...                      ...                      ...               ...   
9445                       0                        0                 0   
9446                       0                        0                 0   
9447                       0                        0                 0   
9448                       0                        0                 0   
9449                       0                        0                 0   

      PC_Twentieth Century Fox_x  PC_Walt Disney Pictures_x  autumn_x  \
0                              0                          0         0   
1                              0                          0         0   
2                              0                          0         0   
3                              0                          0         0   
4                              0                          0         0   
...                          ...                        ...       ...   
9445                           0                          0         0   
9446                           0                          0         0   
9447                           0                          0         0   
9448                           0                          0         0   
9449                           0                          0         0   

      spring_x  summer_x  winter_x  YEAR_2016_x  YEAR_2017_x  YEAR_2018_x  \
0            1         0         0            0            0            0   
1            0         1         0            0            1            0   
2            0         1         0            0            1            0   
3            0         1         0            0            1            0   
4            0         1         0            0            1            0   
...        ...       ...       ...          ...          ...          ...   
9445     

In [3]:
# Delete the useless columns
df = df.drop(["title_x","title_y",
             "budget_x","budget_y"],axis = 1)
pd.set_option("display.max_columns", None)
df

duration_x  LANGUAGE_English_x  LANGUAGE_Chinese_x  LANGUAGE_Korean_x  \
0       0.421053                   1                   0                  0   
1       0.488722                   1                   1                  0   
2       0.488722                   1                   1                  0   
3       0.180451                   1                   0                  0   
4       0.180451                   1                   0                  0   
...          ...                 ...                 ...                ...   
9445    0.481203                   1                   0                  0   
9446    0.481203                   1                   0                  0   
9447    0.481203                   1                   0                  0   
9448    0.481203                   1                   0                  0   
9449    0.481203                   1                   0                  0   

      PC_Marvel Studios_x  PC_Columbia Pictures_x  PC_Universal Pictures_x  \
0                       0                       0                        0   
1                       0                       0                        0   
2                       0                       0                        0   
3                       0                       0                        0   
4                       0                       0                        0   
...                   ...                     ...                      ...   
9445                    0                       0                        0   
9446                    0                       0                        0   
9447                    0                       0                        0   
9448                    0                       0                        0   
9449                    0                       0                        0   

      PC_Warner Bros_x  PC_Twentieth Century Fox_x  PC_Walt Disney Pictures_x  \
0                    1                           0                          0   
1                    1                           0                          0   
2                    1                           0                          0   
3                    0                           0                          0   
4                    0                           0                          0   
...                ...                         ...                        ...   
9445                 0                           0                          0   
9446                 0                           0                          0   
9447                 0                           0                          0   
9448                 0                           0                          0   
9449                 0                           0                          0   

      autumn_x  spring_x  summer_x  winter_x  YEAR_2016_x  YEAR_2017_x  \
0            0         1         0         0            0            0   
1            0         0         1         0            0            1   
2            0         0         1         0            0            1   
3            0         0         1         0            0            1   
4            0         0         1         0            0            1   
...        ...       ...       ...       ...          ...          ...   
9445         0         1         0         0            0            0   
9446         0         1         0         0            0            0   
9447         0         1         0         0            0            0   
9448         0         1         0         0            0            0   
9449         0         1         0         0            0            0   

      YEAR_2018_x  YEAR_2019_x  YEAR_2020_x  GENRE_Action_x  \
0               0            1            0               1   
1               0            0            0               1   
2               0            0            0               1   
3     

In [4]:
# Set the features and target variable for the model
y = df["profit_xy"]
x = df.drop(["profit_xy"],axis = 1)

### Model define

In [16]:
# Native Bayes model which returns AUC
def native_bayes(x_train, y_train, x_test, y_test):
    classifier_nb = GaussianNB()
    classifier_nb.fit(x_train, y_train)
    y_pred_nb = classifier_nb.predict(x_test)
    y_score_nb = classifier_nb.predict_proba(x_test)
    
    fpr_nb, tpr_nb, threshold = roc_curve(y_test, y_score_nb[:, 1])
    auc_nb = auc(fpr_nb, tpr_nb)
    return auc_nb

In [17]:
# Random Forest Classifier which returns AUC
def random_forest(x_train, y_train, x_test, y_test):
    classifier_rf=RandomForestClassifier(n_estimators=100)
    classifier_rf.fit(x_train,y_train)
    y_pred_rf = classifier_rf.predict(x_test)
    y_score_rf = classifier_rf.predict_proba(x_test)
    
    fpr_rf, tpr_rf, threshold = roc_curve(y_test, y_score_rf[:, 1])
    auc_rf = auc(fpr_rf, tpr_rf)
    return auc_rf

### Cross-Validation Building

In [18]:
# Native Bayes
def cross_validate_nb(df, num_folds = 10):
    """
    Performs K fold cross validation using native bayes model.
    Returns an array containing scores from each iteration.
    
    """
    
    # Gets the row numbers for the training folds and the validaition folds for each iteration.
    # tf_indices -> indices for training folds for each iteration
    # vf_indices -> indices for validation folds for each iteration
    tf_indices, vf_indices = get_fold_indices(df, num_folds)

    # Keeps track of the scores for each iteration of cross validation.
    cross_validation_auc_nb = []

    # Performs cross validation
    for i in range(len(tf_indices)):  

        # Training and validation data for the current iteration/
        train_fold_df = df.iloc[tf_indices[i]]
        validation_fold_df = df.iloc[vf_indices[i]]

        # X and y data for training fold
        x_train = train_fold_df.drop("profit_xy", axis = 1).to_numpy()
        y_train = train_fold_df["profit_xy"].to_numpy()

        # X and y data for validation fold
        x_test = validation_fold_df.drop("profit_xy", axis = 1).to_numpy()
        y_test = validation_fold_df["profit_xy"].to_numpy()

        # Run the model, get the prediction and get the auc
        auc_nb = native_bayes(x_train, y_train, x_test, y_test)
        cross_validation_auc_nb.append(auc_nb)


    return cross_validation_auc_nb

In [19]:
# Random Forest Classifier
def cross_validate_rf(df, num_folds = 10):
    """
    Performs K fold cross validation using native bayes model.
    Returns an array containing scores from each iteration.
    
    """
    
    # Gets the row numbers for the training folds and the validaition folds for each iteration.
    # tf_indices -> indices for training folds for each iteration
    # vf_indices -> indices for validation folds for each iteration
    tf_indices, vf_indices = get_fold_indices(df, num_folds)

    # Keeps track of the scores for each iteration of cross validation.
    cross_validation_auc_rf = []

    # Performs cross validation
    for i in range(len(tf_indices)):  

        # Training and validation data for the current iteration/
        train_fold_df = df.iloc[tf_indices[i]]
        validation_fold_df = df.iloc[vf_indices[i]]

        # X and y data for training fold
        x_train = train_fold_df.drop("profit_xy", axis = 1).to_numpy()
        y_train = train_fold_df["profit_xy"].to_numpy()

        # X and y data for validation fold
        x_test = validation_fold_df.drop("profit_xy", axis = 1).to_numpy()
        y_test = validation_fold_df["profit_xy"].to_numpy()

        # Run the model, get the prediction and get the auc
        auc_rf = random_forest(x_train, y_train, x_test, y_test)
        cross_validation_auc_rf.append(auc_rf)

    return cross_validation_auc_rf

### Model Appllication

In [20]:
#  Split the data into train set and test set 
df_train, df_test = get_train_test_split(df, train_size = 0.8)

# Re-calculates the indices because pandas normally keeps the original indices when you take a section of a dataframe
df_train.reset_index(inplace=True, drop=True) 
df_test.reset_index(inplace=True, drop=True)

# Native Bayes
cross_validation_auc_nb = cross_validate_nb(df_train)
average_cv_auc_nb = round(100 * sum(cross_validation_auc_nb)/len(cross_validation_auc_nb),2)

print(f"Cross validation auc(%) for Native Bayes Model = {np.around(100 * np.array(cross_validation_auc_nb),2)}")
print(f"Average cross validation auc for Native Bayes Model = {average_cv_auc_nb}%")

# Random Forest
cross_validation_auc_rf = cross_validate_rf(df_train)
average_cv_auc_rf = round(100 * sum(cross_validation_auc_rf)/len(cross_validation_auc_rf),2)

print(f"Cross validation auc(%) for Random Forest Model = {np.around(100 * np.array(cross_validation_auc_rf),2)}")
print(f"Average cross validation auc for Random Forest Model = {average_cv_auc_rf}%")


Cross validation auc(%) for Native Bayes Model = [71.03 67.72 70.31 73.68 69.48 72.81 72.58 73.86 70.22 67.48]
Average cross validation auc for Native Bayes Model = 70.92%
Cross validation auc(%) for Random Forest Model = [94.43 94.   94.14 94.9  93.81 95.22 95.35 94.46 92.82 93.19]
Average cross validation auc for Random Forest Model = 94.23%
